<span style="font-family: 'Palatino Linotype', 'Book Antiqua', Palatino, serif; font-size: 24px;">
The cells below don't need to be run again. They were used to create the cleaned_data.csv file.</span>

<br><br>
<span style="font-family: 'Palatino Linotype', 'Book Antiqua', Palatino, serif; font-size: 18px;">
The data is loaded, and unnecessary columns are removed. 
Empty values are replaced by NaN values so all missing/error values have the same name.</span>

In [1]:
import re
import pandas as pd
import numpy as np

# Import the data
df = pd.read_csv('all_data.csv', delimiter=';')
pd.set_option('display.max_columns', None)
#pd.set_option('display.max_rows', None)

# drop Name and SSN columns
df = df.drop(['Name', 'SSN'], axis=1)

# Replace all empty cells with NaN in the entire dataframe
df.replace([None], np.nan, inplace=True)

C:\Users\timow\AppData\Local\Temp\ipykernel_30840\1390940802.py:6: DtypeWarning: Columns (27) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('all_data.csv', delimiter=';')


<br><br>
<span style="font-family: 'Palatino Linotype', 'Book Antiqua', Palatino, serif; font-size: 18px;">
First we order the data by Customer_ID and Month. That way we have a nice overview of 12 consecutive data points that belong to a single customer.</span>

In [2]:
# Order DataFrame by customer id and month
month_to_num = {'January': 1, 'February': 2, 'March': 3, 'April': 4, 'May': 5, 
                'June': 6, 'July': 7, 'August': 8, 'September': 9, 
                'October': 10, 'November': 11, 'December': 12}

df["temp_Month"] = df["Month"].map(month_to_num)
df.sort_values(["Customer_ID", "temp_Month"], inplace=True)
df.drop("temp_Month", axis=1, inplace=True)
df.reset_index(drop=True, inplace=True)

<br><br>
<span style="font-family: 'Palatino Linotype', 'Book Antiqua', Palatino, serif; font-size: 18px;">
The numeric columns are cleaned from unwanted (special) characters, and are all transformed into the correct data type.  
It was decided that error values of the Occupation column will be replaced by the most occurring value of that customer.  
Then certain error values are replaced with a uniform value ("Not available" for strings, Nan for numeric values) so we can easily identify them in the future.  
The Payment_Behaviour column is split into two columns. If this is not used in the future, remove from code!</span>

In [3]:
# Define columns with numeric values
numeric_column_names = ["Age", "Annual_Income", "Monthly_Inhand_Salary", "Num_Bank_Accounts", 
"Num_Credit_Card", "Interest_Rate", "Num_of_Loan", "Delay_from_due_date", 
"Num_of_Delayed_Payment", "Changed_Credit_Limit", "Num_Credit_Inquiries", 
"Outstanding_Debt", "Credit_Utilization_Ratio", "Total_EMI_per_month", 
"Amount_invested_monthly", "Monthly_Balance"]

# Clean numeric columns from unwanted characters 
special_chars = set()
special_char_regex = r'[^0-9]'
for column in numeric_column_names:
    df[column] = df[column].astype(str).str.replace(',', '.').str.replace('_', '') # Perform replacements
    df[column] = pd.to_numeric(df[column], errors='coerce').round(2) # Convert to numeric


# Custom cleaning for specific columns
df["Occupation"] = df.groupby("Customer_ID")["Occupation"].transform(lambda x: x.mode()[0] if x.mode().size > 0 else np.nan) # Replaces NaN values with the mode of that customer
df['Amount_invested_monthly'] = df['Amount_invested_monthly'].replace(10000, np.nan)
df['Credit_Mix'] = df['Credit_Mix'].replace("_", "Not available")
df['Payment_of_Min_Amount'] = df['Payment_of_Min_Amount'].replace("NM", "Not available")
df["Payment_Behaviour"] = df["Payment_Behaviour"].replace("!@9#%8", "Not available")


# Split 'Payment_Behaviour' column into two new columns
df[['Behaviour_Spending_Level', 'Behaviour_Value_Size']] = df['Payment_Behaviour'].str.split('_(?=[A-Z])', n=1, expand=True)
df.loc[df['Payment_Behaviour'] == 'Not available', ['Behaviour_Spending_Level', 'Behaviour_Value_Size']] = 'Not available'

# Set new columns next to 'Payment_Behaviour'
pb_index = df.columns.get_loc('Payment_Behaviour')
df = df[df.columns.tolist()[:pb_index+1] + ['Behaviour_Spending_Level', 'Behaviour_Value_Size'] + df.columns.tolist()[pb_index+1:-2]]

<br><br>
<span style="font-family: 'Palatino Linotype', 'Book Antiqua', Palatino, serif; font-size: 18px;">
In columns where negative values are not allowed/possible, negative values will be replaced by the mode of the positive values of the customer.</span>

In [4]:
# Define columns with non-negative numeric values
non_negative_column_names = numeric_column_names.copy()
non_negative_column_names.remove("Delay_from_due_date")
non_negative_column_names.remove("Changed_Credit_Limit")

# Replace negative values  with the mode (of the positive) values of the customer
for column in non_negative_column_names:
    mode_values = df.groupby('Customer_ID')[column].transform(lambda x: x[x >= 0].mode()[0])
    df.loc[df[column] < 0, column] = mode_values[df[column] < 0]

Age
Annual_Income
Monthly_Inhand_Salary
Num_Bank_Accounts
Num_Credit_Card
Interest_Rate
Num_of_Loan
Num_of_Delayed_Payment
Num_Credit_Inquiries
Outstanding_Debt
Credit_Utilization_Ratio
Total_EMI_per_month
Amount_invested_monthly
Monthly_Balance


<br><br>
<span style="font-family: 'Palatino Linotype', 'Book Antiqua', Palatino, serif; font-size: 18px;">
To filter out error/impossible values from certain columns, an upper limit is selected. Every value above this upper limit will be replaced by the mode of the other values of that customer.  
It was decided that if a value of a column occurs more than once for a customer, this value is legitimate and thus not an error/impossible value.  
By sorting these values for each numeric column (with a few exceptions), the largest value was selected as an upper limit for these columns.  The code below was used to find these limits.  
After investigation, it seems like Amount_invested_monthly and Monthly_Balance do not have error values. So it was not needed to assign those columns an upper limit.</span>

In [ ]:
def highest_values(column_name):
    """
    This function will give you the 10 highest values of a column where 
    the value appears more than once for a customer.
    This will identify the highest values that are most likely not error values.
    """
    income_counts = df.groupby(['Customer_ID', column_name]).size().reset_index(name='count')
    income_counts_filtered = income_counts[income_counts['count'] >= 1]
    income_counts_filtered_sorted = income_counts_filtered.sort_values(column_name, ascending=False)
    top_10_incomes = income_counts_filtered_sorted.head(10)
    print(top_10_incomes)

highest_values("Amount_invested_monthly")

<span style="font-family: 'Palatino Linotype', 'Book Antiqua', Palatino, serif; font-size: 18px;">
For the selected columns, the upper limits are defined and any value above those limits are replaced by the mode value of that customer.
</span>

In [5]:
# Replace values above the upper limit with the mode value of that customer
upper_limit_dict = {"Age": 56, "Annual_Income": 179987.28, "Monthly_Inhand_Salary": 15204.63,
                  "Num_Bank_Accounts": 11, "Num_Credit_Card": 11, "Interest_Rate": 34,
                  "Num_of_Loan": 9, "Delay_from_due_date": 67, "Num_of_Delayed_Payment": 28,
                  "Changed_Credit_Limit": 34.21, "Num_Credit_Inquiries": 17, "Outstanding_Debt": 4998.07,
                  "Credit_Utilization_Ratio": 43.06, "Total_EMI_per_month": 1841.35}

for column, limit in upper_limit_dict.items():
    # Calculate the mode value for each customer_id in the specified column
    mode_values = df.groupby('Customer_ID')[column].transform(lambda x: x.mode()[0])
    # Replace values above the limit with the corresponding mode value
    df.loc[df[column] > limit, column] = mode_values[df[column] > limit]

<br><br>
<span style="font-family: 'Palatino Linotype', 'Book Antiqua', Palatino, serif; font-size: 18px;">
Credit_History_Age is transformed from its string format to a numeric representation of the total number of months.  
After this, the missing values are replaced by the correct ones with the following logic:
First, every missing value in January is replaced by the correct value by looking at the subsequent months until a non-NaN value is found. The missing value in January is then calculated based on the found value and the distance between those two months.
After this, every value in January is correct, and every subsequent month for each customer can easily be calculated and inserted into the dataset.
</span>

In [6]:
def total_months(str):
    """
    This function will convert a string of the format "{x} years and {y} months" 
    to the total number of months.
    """
    if str is np.nan:
        return np.nan
    
    numbers = re.findall(r'\d+', str)
    if len(numbers) == 2:
        return int(numbers[0]) * 12 + int(numbers[1])
    else:
        return np.nan

df['Credit_History_Age'] = df['Credit_History_Age'].apply(total_months)


# Replace missing values with the correct values based on existing data
for idx in range(len(df)):
    if df.loc[idx, "Month"] == "January":
        if pd.isna(df.loc[idx, "Credit_History_Age"]):
            # If the first month's value is NaN, find the next non-NaN value
            distance = 0
            current_idx = idx
            while current_idx < len(df) and pd.isna(df.loc[current_idx, "Credit_History_Age"]):
                distance += 1
                current_idx += 1

            # Calculate the new value for January
            new_value = df.loc[current_idx, 'Credit_History_Age'] - distance
            df.at[idx, 'Credit_History_Age'] = new_value
        
        # Set the starting value for the customer
        starting_value = df.loc[idx, 'Credit_History_Age']
        current_customer_id = df.loc[idx, 'Customer_ID']
        current_idx = idx + 1

        # Increment the value for each subsequent month
        while current_idx < len(df) and df.loc[current_idx, 'Customer_ID'] == current_customer_id:
            starting_value += 1
            df.at[current_idx, 'Credit_History_Age'] = starting_value
            current_idx += 1

<br><br>
<span style="font-family: 'Palatino Linotype', 'Book Antiqua', Palatino, serif; font-size: 18px;">
Finally, the data is stored in a new csv file for easy access. 
</span>

In [ ]:
df.to_csv('cleaned_data.csv', index=False)

<br><br>
<span style="font-family: 'Palatino Linotype', 'Book Antiqua', Palatino, serif; font-size: 18px;">
Below are some cells for checking the new dataset. 
</span>

In [7]:
for column in non_negative_column_names:
    print(f"{column}_min: {df[column].min()}")
    print(f"{column}_max: {df[column].max()}")

Age_min: 14
Age_max: 56
Annual_Income_min: 7005.93
Annual_Income_max: 179987.28
Monthly_Inhand_Salary_min: 303.65
Monthly_Inhand_Salary_max: 15204.63
Num_Bank_Accounts_min: 0
Num_Bank_Accounts_max: 11
Num_Credit_Card_min: 0
Num_Credit_Card_max: 11
Interest_Rate_min: 1
Interest_Rate_max: 34
Num_of_Loan_min: 0
Num_of_Loan_max: 9
Num_of_Delayed_Payment_min: 0.0
Num_of_Delayed_Payment_max: 28.0
Num_Credit_Inquiries_min: 0.0
Num_Credit_Inquiries_max: 17.0
Outstanding_Debt_min: 0.23
Outstanding_Debt_max: 4998.07
Credit_Utilization_Ratio_min: 20.0
Credit_Utilization_Ratio_max: 43.06
Total_EMI_per_month_min: 0.0
Total_EMI_per_month_max: 1841.35
Amount_invested_monthly_min: 0.0
Amount_invested_monthly_max: 1977.33
Monthly_Balance_min: 0.01
Monthly_Balance_max: 1606.52


In [9]:
nan_columns = df.isna().any()
print(nan_columns)

ID                          False
Customer_ID                 False
Month                       False
Age                         False
Occupation                  False
Annual_Income               False
Monthly_Inhand_Salary        True
Num_Bank_Accounts           False
Num_Credit_Card             False
Interest_Rate               False
Num_of_Loan                 False
Type_of_Loan                 True
Delay_from_due_date         False
Num_of_Delayed_Payment       True
Changed_Credit_Limit         True
Num_Credit_Inquiries         True
Credit_Mix                  False
Outstanding_Debt            False
Credit_Utilization_Ratio    False
Credit_History_Age          False
Payment_of_Min_Amount       False
Total_EMI_per_month         False
Amount_invested_monthly      True
Payment_Behaviour           False
Behaviour_Spending_Level    False
Behaviour_Value_Size        False
Monthly_Balance              True
Credit_Score                 True
dtype: bool


In [8]:
df.describe()

,Age,Annual_Income,Monthly_Inhand_Salary,Num_Bank_Accounts,Num_Credit_Card,Interest_Rate,Num_of_Loan,Delay_from_due_date,Num_of_Delayed_Payment,Changed_Credit_Limit,Num_Credit_Inquiries,Outstanding_Debt,Credit_Utilization_Ratio,Credit_History_Age,Total_EMI_per_month,Amount_invested_monthly,Monthly_Balance
count,150000.000000,150000.000000,127500.000000,150000.000000,150000.000000,150000.000000,150000.000000,150000.000000,139500.000000,146850.000000,147000.000000,150000.000000,150000.000000,150000.00000,150000.000000,136770.000000,148238.000000
mean,33.479060,50505.123465,4190.115094,5.368953,5.533660,14.532080,3.532880,21.063400,13.331570,10.382420,6.281381,1426.220376,32.168156,223.20688,113.159958,195.305032,402.704999
std,10.767158,38299.358243,3180.489626,2.593830,2.068672,8.741316,2.446352,14.860154,6.273371,6.780487,3.957040,1155.127101,5.037935,99.71438,144.487084,198.599626,213.843455
min,14.000000,7005.930000,303.650000,0.000000,0.000000,1.000000,0.000000,-5.000000,0.000000,-6.490000,0.000000,0.230000,20.000000,1.00000,0.000000,0.000000,0.010000
25%,25.000000,19342.972500,1625.270000,3.000000,4.000000,7.000000,2.000000,10.000000,9.000000,5.330000,3.000000,566.072500,27.980000,146.00000,29.920000,72.230000,270.230000
50%,33.000000,36999.705000,3091.000000,5.000000,5.000000,13.000000,3.000000,18.000000,14.000000,9.410000,6.000000,1166.155000,32.190000,222.00000,67.950000,128.985000,336.800000
75%,42.000000,71683.470000,5948.450000,7.000000,7.000000,20.000000,5.000000,28.000000,18.000000,14.840000,9.000000,1945.962500,36.380000,304.00000,152.400000,236.840000,470.455000
max,56.000000,179987.280000,15204.630000,11.000000,11.000000,34.000000,9.000000,67.000000,28.000000,34.210000,17.000000,4998.070000,43.060000,408.00000,1841.350000,1977.330000,1606.520000


The sum of Monthly_Inhand_Salary and Total_EMI_per_month will always sum to the same amount for a customer (with an error margin of 0.01). As there are no NaN values in total_EMI, we can use this to calculate the missing Monthly_Inhand_Salary values.

Age still needs to be fixed.

For Type_of_Loan, these are the possible loans that a value might contain:

'Debt Consolidation Loan', 'Student Loan', 'Personal Loan', 'Auto Loan', 'Home Equity Loan', 'Mortgage Loan', 'Payday Loan', 'Not Specified', 'Credit-Builder Loan', 'Type_of_Loan'

<br><br>
<span style="font-family: 'Palatino Linotype', 'Book Antiqua', Palatino, serif; font-size: 18px;">
By running the code below you create a df where the entries that have a NaN value in any column are removed completely.
</span>

In [8]:
# Remove any row where any value (except credit_score) is NaN
import pandas as pd
df = pd.read_csv('cleaned_data.csv')
columns_to_consider = [col for col in df.columns if col != 'credit_score']
df_cleaned_not_cs = df.dropna(subset=columns_to_consider)
df_cleaned_not_cs.describe()

,Age,Annual_Income,Monthly_Inhand_Salary,Num_Bank_Accounts,Num_Credit_Card,Interest_Rate,Num_of_Loan,Delay_from_due_date,Num_of_Delayed_Payment,Changed_Credit_Limit,Num_Credit_Inquiries,Outstanding_Debt,Credit_Utilization_Ratio,Credit_History_Age,Total_EMI_per_month,Amount_invested_monthly,Monthly_Balance
count,60588.000000,60588.000000,60588.000000,60588.000000,60588.000000,60588.000000,60588.000000,60588.000000,60588.000000,60588.000000,60588.000000,60588.000000,60588.000000,60588.000000,60588.000000,60588.000000,60588.000000
mean,32.914917,48265.258968,4013.461548,5.523057,5.647059,15.223757,3.973229,21.993349,13.742078,10.743422,6.114775,1508.033353,32.111529,212.977471,116.231279,181.873769,381.897365
std,10.652305,37162.179214,3094.021006,2.577297,2.079052,8.872711,2.220430,15.225586,6.240910,6.902401,3.907149,1188.364554,5.010567,99.798322,125.852792,177.006219,195.065595
min,14.000000,7005.930000,303.650000,0.000000,0.000000,1.000000,1.000000,-5.000000,0.000000,-6.480000,0.000000,0.230000,20.800000,1.000000,4.460000,0.000000,0.130000
25%,24.000000,18729.060000,1570.835000,4.000000,4.000000,8.000000,2.000000,10.000000,9.000000,5.610000,3.000000,606.470000,27.940000,134.000000,40.100000,69.740000,264.700000
50%,33.000000,35664.720000,2979.520000,6.000000,6.000000,15.000000,4.000000,19.000000,14.000000,9.700000,6.000000,1240.700000,32.150000,211.000000,75.290000,123.910000,325.120000
75%,41.000000,68412.510000,5684.040000,8.000000,7.000000,21.000000,6.000000,29.000000,19.000000,15.480000,9.000000,2132.902500,36.320000,291.000000,156.240000,223.040000,442.982500
max,56.000000,179987.280000,15204.630000,11.000000,11.000000,34.000000,9.000000,67.000000,28.000000,34.200000,17.000000,4998.070000,43.060000,404.000000,1804.000000,1607.300000,1552.950000


In [3]:
# Remove rows where any value is NaN
import pandas as pd
df = pd.read_csv('cleaned_data.csv')
df_cleaned = df.dropna()
df_cleaned.describe()

,Age,Annual_Income,Monthly_Inhand_Salary,Num_Bank_Accounts,Num_Credit_Card,Interest_Rate,Num_of_Loan,Delay_from_due_date,Num_of_Delayed_Payment,Changed_Credit_Limit,Num_Credit_Inquiries,Outstanding_Debt,Credit_Utilization_Ratio,Credit_History_Age,Total_EMI_per_month,Amount_invested_monthly,Monthly_Balance
count,60588.000000,60588.000000,60588.000000,60588.000000,60588.000000,60588.000000,60588.000000,60588.000000,60588.000000,60588.000000,60588.000000,60588.000000,60588.000000,60588.000000,60588.000000,60588.000000,60588.000000
mean,32.914917,48265.258968,4013.461548,5.523057,5.647059,15.223757,3.973229,21.993349,13.742078,10.743422,6.114775,1508.033353,32.111529,212.977471,116.231279,181.873769,381.897365
std,10.652305,37162.179214,3094.021006,2.577297,2.079052,8.872711,2.220430,15.225586,6.240910,6.902401,3.907149,1188.364554,5.010567,99.798322,125.852792,177.006219,195.065595
min,14.000000,7005.930000,303.650000,0.000000,0.000000,1.000000,1.000000,-5.000000,0.000000,-6.480000,0.000000,0.230000,20.800000,1.000000,4.460000,0.000000,0.130000
25%,24.000000,18729.060000,1570.835000,4.000000,4.000000,8.000000,2.000000,10.000000,9.000000,5.610000,3.000000,606.470000,27.940000,134.000000,40.100000,69.740000,264.700000
50%,33.000000,35664.720000,2979.520000,6.000000,6.000000,15.000000,4.000000,19.000000,14.000000,9.700000,6.000000,1240.700000,32.150000,211.000000,75.290000,123.910000,325.120000
75%,41.000000,68412.510000,5684.040000,8.000000,7.000000,21.000000,6.000000,29.000000,19.000000,15.480000,9.000000,2132.902500,36.320000,291.000000,156.240000,223.040000,442.982500
max,56.000000,179987.280000,15204.630000,11.000000,11.000000,34.000000,9.000000,67.000000,28.000000,34.200000,17.000000,4998.070000,43.060000,404.000000,1804.000000,1607.300000,1552.950000


In [9]:
# Check if both DataFrames are the same
same_shape = df_cleaned.shape == df_cleaned_not_cs.shape
same_columns = all(df_cleaned.columns == df_cleaned_not_cs.columns)
same_values = df_cleaned.equals(df_cleaned_not_cs)

are_same = same_shape and same_columns and same_values
print(f"Are df_cleaned and df_cleaned_not_cs the same? {are_same}")

Are df_cleaned and df_cleaned_not_cs the same? True


<span style="font-family: 'Palatino Linotype', 'Book Antiqua', Palatino, serif; font-size: 18px;">
As it turns out, any entry that has a NaN value in any column except Credit_Score, also has a NaN value in Credit_Score. 
These entries would be removed anyway, so there is no point in replacing the missing values of those other columns.
Unless we plan on replacing the missing values in Credit_Score, df_cleaned contains the final dataframe we will work with.
</span>

In [ ]:
# save cleaned data
df_cleaned.to_csv('cleaned_data.csv', index=False)